```{contents}
```

## Reshaping & Pivot

In [1]:

import pandas as pd

# Sample loan dataset
data = {
    'application_id': ['A1','A2','A3','A4'],
    'customer_id': ['C1','C2','C3','C4'],
    'loan_type': ['Business Loan', 'Car Loan', 'Education Loan', 'Car Loan'],
    'loan_status': ['Approved','Approved','Approved','Declined'],
    'loan_amount_requested': [604000, 100000, 431000, 324000],
    'monthly_income': [34700, 51600, 14800, 28800]
}

df = pd.DataFrame(data)
df


,application_id,customer_id,loan_type,loan_status,loan_amount_requested,monthly_income
0,A1,C1,Business Loan,Approved,604000,34700
1,A2,C2,Car Loan,Approved,100000,51600
2,A3,C3,Education Loan,Approved,431000,14800
3,A4,C4,Car Loan,Declined,324000,28800




---

### Reshaping using `melt`

* `melt` transforms a **wide dataframe into long format**.



In [2]:
# Melt loan_amount_requested and monthly_income into a single column
df_melted = pd.melt(df,
                    id_vars=['application_id','customer_id','loan_type','loan_status'],
                    value_vars=['loan_amount_requested','monthly_income'],
                    var_name='metric',
                    value_name='value')

df_melted


,application_id,customer_id,loan_type,loan_status,metric,value
0,A1,C1,Business Loan,Approved,loan_amount_requested,604000
1,A2,C2,Car Loan,Approved,loan_amount_requested,100000
2,A3,C3,Education Loan,Approved,loan_amount_requested,431000
3,A4,C4,Car Loan,Declined,loan_amount_requested,324000
4,A1,C1,Business Loan,Approved,monthly_income,34700
5,A2,C2,Car Loan,Approved,monthly_income,51600
6,A3,C3,Education Loan,Approved,monthly_income,14800
7,A4,C4,Car Loan,Declined,monthly_income,28800




**Result:** Columns `loan_amount_requested` and `monthly_income` are stacked under `metric` and `value`.

---

### Pivoting using `pivot`

* `pivot` transforms **long data into wide format**.



In [3]:

# Example: pivot metrics back
df_pivot = df_melted.pivot(index='application_id',
                           columns='metric',
                           values='value').reset_index()

df_pivot


metric,application_id,loan_amount_requested,monthly_income
0,A1,604000,34700
1,A2,100000,51600
2,A3,431000,14800
3,A4,324000,28800




**Result:** Two columns (`loan_amount_requested` and `monthly_income`) are restored for each `application_id`.

---

### Pivot Table using `pivot_table`

* `pivot_table` allows **aggregation while pivoting**.



In [5]:
# Average loan amount per loan_type and loan_status
df_pivot_table = df.pivot_table(index='loan_type',
                                columns='loan_status',
                                values='loan_amount_requested',
                                aggfunc='mean',
                                fill_value=0)

df_pivot_table


loan_status,Approved,Declined
loan_type,,
Business Loan,604000.0,0.0
Car Loan,100000.0,324000.0
Education Loan,431000.0,0.0


**Explanation:**

* `index` → rows (group by `loan_type`)
* `columns` → new columns (`loan_status`)
* `values` → numeric column to aggregate (`loan_amount_requested`)
* `aggfunc` → aggregation function (`mean`)
* `fill_value` → replace NaNs with 0

---

### Reshaping with `stack` and `unstack`


In [6]:
# Set multi-index
df_multi = df.set_index(['loan_type','loan_status'])
df_multi

# Unstack (pivot level from index to columns)
df_unstacked = df_multi['loan_amount_requested'].unstack(level=1)
df_unstacked

# Stack back
df_stacked = df_unstacked.stack()
df_stacked


loan_type       loan_status
Business Loan   Approved       604000.0
Car Loan        Approved       100000.0
                Declined       324000.0
Education Loan  Approved       431000.0
dtype: float64



**Explanation:**

* `stack()` → moves columns to inner level of index (long format)
* `unstack()` → moves inner index level to columns (wide format)

---

**Summary **

| Method        | Purpose                | Example                                                                                                    |
| ------------- | ---------------------- | ---------------------------------------------------------------------------------------------------------- |
| `melt`        | Wide → Long            | `pd.melt(df, value_vars=[...])`                                                                            |
| `pivot`       | Long → Wide            | `df.pivot(index, columns, values)`                                                                         |
| `pivot_table` | Pivot with aggregation | `df.pivot_table(index='loan_type', columns='loan_status', values='loan_amount_requested', aggfunc='mean')` |
| `stack`       | Columns → inner index  | `df.stack()`                                                                                               |
| `unstack`     | Inner index → columns  | `df.unstack()`                                                                                             |

